### 1. Library Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.preprocessing import KBinsDiscretizer, StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity, calculate_kmo

### 2 Cargar y Mostrar Dataset

In [3]:

df = pd.read_csv("./movies.csv", encoding="ISO-8859-1")

df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         10000 non-null  int64  
 1   budget                     10000 non-null  int64  
 2   genres                     9947 non-null   object 
 3   homePage                   4193 non-null   object 
 4   productionCompany          9543 non-null   object 
 5   productionCompanyCountry   8720 non-null   object 
 6   productionCountry          9767 non-null   object 
 7   revenue                    10000 non-null  float64
 8   runtime                    10000 non-null  int64  
 9   video                      9514 non-null   object 
 10  director                   9926 non-null   object 
 11  actors                     9920 non-null   object 
 12  actorsPopularity           9913 non-null   object 
 13  actorsCharacter            9953 non-null   obje

,id,budget,revenue,runtime,popularity,voteAvg,voteCount,genresAmount,productionCoAmount,productionCountriesAmount,actorsAmount
count,10000.000000,1.000000e+04,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,249876.829300,1.855163e+07,5.673793e+07,100.268100,51.393907,6.483490,1342.381800,2.596500,3.171400,1.751000,2147.666600
std,257380.109004,3.662669e+07,1.495854e+08,27.777829,216.729552,0.984274,2564.196637,1.154565,2.539738,3.012093,37200.075802
min,5.000000,0.000000e+00,0.000000e+00,0.000000,4.258000,1.300000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,12286.500000,0.000000e+00,0.000000e+00,90.000000,14.577750,5.900000,120.000000,2.000000,2.000000,1.000000,13.000000
50%,152558.000000,5.000000e+05,1.631245e+05,100.000000,21.905500,6.500000,415.000000,3.000000,3.000000,1.000000,21.000000
75%,452021.750000,2.000000e+07,4.479661e+07,113.000000,40.654000,7.200000,1316.000000,3.000000,4.000000,2.000000,36.000000
max,922260.000000,3.800000e+08,2.847246e+09,750.000000,11474.647000,10.000000,30788.000000,16.000000,89.000000,155.000000,919590.000000


### 3 Detectar columnas con datos "|"

In [ ]:
numeric_cols = ['budget', 'revenue', 'runtime', 'popularity', 'voteAvg', 'voteCount', 
            'actorsPopularity', 'actorsAmount', 'castWomenAmount', 'castMenAmount']

# Identificar columnas con "|"
for col in numeric_cols:
    if df[col].astype(str).str.contains("\|").any():
        print(f"La columna '{col}' contiene valores separados por '|'.") 

        # Dividir valores por "|", convertir a float y tomar el promedio.
        df[col] = df[col].astype(str).apply(lambda x: np.mean([float(v) for v in x.split("|") if v.replace('.', '', 1).isdigit()]))


# Verificar NANS
for col in numeric_cols:
    non_numeric_values = df[col].astype(str).str.contains("[a-zA-Z]", regex=True).sum()
    if non_numeric_values > 0:
        print(f"La columna '{col}' tiene {non_numeric_values} valores no numéricos.")
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)



La columna 'actorsPopularity' tiene 123 valores no numéricos.
La columna 'castWomenAmount' tiene 37 valores no numéricos.
La columna 'castMenAmount' tiene 162 valores no numéricos.


<>:6: DeprecationWarning: invalid escape sequence '\|'
<>:6: DeprecationWarning: invalid escape sequence '\|'
C:\Users\pablo\AppData\Local\Temp\ipykernel_17800\1961260763.py:6: DeprecationWarning: invalid escape sequence '\|'
  if df[col].astype(str).str.contains("\|").any():
